In [18]:
from datetime import datetime
import os

PROJECT = "qwiklabs-gcp-ml-49b827b781ab"  # REPLACE WITH YOUR PROJECT ID
BUCKET = "qwiklabs-gcp-ml-49b827b781ab"  # REPLACE WITH YOUR BUCKET NAME
REGION = "us-central1"  # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# Do not change these
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["IMAGE_URI"] = os.path.join("gcr.io", PROJECT, "mnist_trainer")

In [25]:
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import (
    Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Softmax)

In [26]:
WIDTH = 28
HEIGHT = 28

In [27]:
def scale(image, label):
    """Scales images from a 0-255 int range to a 0-1 float range"""
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.expand_dims(image, -1)
    return image, label

In [28]:
def load_dataset(
        data, training=True, buffer_size=5000, batch_size=100, nclasses=10):
    """Loads MNIST dataset into a tf.data.Dataset"""
    (x_train, y_train), (x_test, y_test) = data
    x = x_train if training else x_test
    y = y_train if training else y_test
    # One-hot encode the classes
    y = tf.keras.utils.to_categorical(y, nclasses)
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(scale).batch(batch_size)
    if training:
        dataset = dataset.shuffle(buffer_size).repeat()
    return dataset

In [59]:
def get_layers(model_type,
               nclasses=10,
               hidden_layer_1_neurons=400,
               hidden_layer_2_neurons=100,
               dropout_rate=0.25,
               num_filters_1=64,
               kernel_size_1=3,
               pooling_size_1=2,
               num_filters_2=32,
               kernel_size_2=3,
               pooling_size_2=2):
    """Constructs layers for a keras model based on a dict of model types."""
    model_layers = {
        'linear': [
            Flatten(),
            Dense(nclasses),
            Softmax()
        ],
        'dnn': [
            Flatten(),
            Dense(hidden_layer_1_neurons, activation='relu'),
            Dense(hidden_layer_2_neurons, activation='relu'),
            Dense(nclasses),
            Softmax()
        ],
        'dnn_dropout': [
            Flatten(),
            Dense(hidden_layer_1_neurons, activation='relu'),
            Dense(hidden_layer_2_neurons, activation='relu'),
            Dropout(dropout_rate),
            Dense(nclasses),
            Softmax()
        ],
        'cnn': [
            Conv2D(num_filters_1, kernel_size=kernel_size_1,
                   activation='relu', input_shape=(WIDTH, HEIGHT, 1)),
            MaxPooling2D(pooling_size_1),
            Conv2D(num_filters_2, kernel_size=kernel_size_2,
                   activation='relu'),
            MaxPooling2D(pooling_size_2),
            Flatten(),
            Dense(hidden_layer_1_neurons, activation='relu'),
            Dense(hidden_layer_2_neurons, activation='relu'),
            Dropout(dropout_rate),
            Dense(nclasses),
            Softmax()
        ]
    }
    return model_layers[model_type]

In [63]:
def build_model(model_type):
    model_layers = get_layers(model_type) 

    """Compiles keras model for image classification."""
    model = Sequential(model_layers)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [70]:
def train_and_evaluate(model, num_epochs, steps_per_epoch, output_dir):

    model = build_model(model_type)

    """Compiles keras model and loads data into it for training."""
    mnist = tf.keras.datasets.mnist.load_data()
    train_data = load_dataset(mnist)
    validation_data = load_dataset(mnist, training=False)

    callbacks = []
    if output_dir:
        tensorboard_callback = TensorBoard(log_dir=output_dir)
        callbacks = [tensorboard_callback]

    history = model.fit(
        train_data,
        validation_data=validation_data,
        epochs=num_epochs,
        steps_per_epoch=steps_per_epoch,
        verbose=2,
        callbacks=callbacks)

    model.summary()

    if output_dir:
        export_path = os.path.join(output_dir, 'keras_export')
        model.save(export_path, save_format='tf')

    return history

In [71]:
model_type = 'linear' # linear / dnn / dnn_dropout / cnn
current_time = datetime.now().strftime("%y%m%d_%H%M%S")
epochs = 5
steps_per_epoch = 50
output_dir = "mnist_trainer/models/{}_{}/".format(model_type, current_time)

model_history = train_and_evaluate(image_model, epochs, steps_per_epoch, output_dir)

Train for 50 steps, validate for 100 steps
Epoch 1/5
50/50 - 5s - loss: 1.7332 - accuracy: 0.5036 - val_loss: 1.1921 - val_accuracy: 0.7780
Epoch 2/5
50/50 - 1s - loss: 0.9730 - accuracy: 0.8144 - val_loss: 0.7970 - val_accuracy: 0.8308
Epoch 3/5
50/50 - 1s - loss: 0.7351 - accuracy: 0.8384 - val_loss: 0.6437 - val_accuracy: 0.8569
Epoch 4/5
50/50 - 1s - loss: 0.6302 - accuracy: 0.8484 - val_loss: 0.5590 - val_accuracy: 0.8700
Epoch 5/5
50/50 - 1s - loss: 0.5363 - accuracy: 0.8692 - val_loss: 0.5074 - val_accuracy: 0.8782
Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_68 (Flatten)         multiple                  0         
_________________________________________________________________
dense_170 (Dense)            multiple                  7850      
_________________________________________________________________
softmax_68 (Softmax)         multiple                

In [74]:
model_type = 'dnn' # linear / dnn / dnn_dropout / cnn
current_time = datetime.now().strftime("%y%m%d_%H%M%S")
epochs = 5
steps_per_epoch = 50
output_dir = "mnist_trainer/models/{}_{}/".format(model_type, current_time)


model_history = train_and_evaluate(image_model, epochs, steps_per_epoch, output_dir)

Train for 50 steps, validate for 100 steps
Epoch 1/5
50/50 - 6s - loss: 0.8251 - accuracy: 0.7696 - val_loss: 0.3950 - val_accuracy: 0.8806
Epoch 2/5
50/50 - 1s - loss: 0.3538 - accuracy: 0.8946 - val_loss: 0.2944 - val_accuracy: 0.9165
Epoch 3/5
50/50 - 1s - loss: 0.2662 - accuracy: 0.9254 - val_loss: 0.2526 - val_accuracy: 0.9265
Epoch 4/5
50/50 - 1s - loss: 0.2447 - accuracy: 0.9312 - val_loss: 0.2160 - val_accuracy: 0.9372
Epoch 5/5
50/50 - 1s - loss: 0.2350 - accuracy: 0.9252 - val_loss: 0.2043 - val_accuracy: 0.9407
Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_89 (Flatten)         multiple                  0         
_________________________________________________________________
dense_221 (Dense)            multiple                  314000    
_________________________________________________________________
dense_222 (Dense)            multiple                

In [76]:
model_type = 'dnn_dropout' # linear / dnn / dnn_dropout / cnn
current_time = datetime.now().strftime("%y%m%d_%H%M%S")
epochs = 5
steps_per_epoch = 50
output_dir = "mnist_trainer/models/{}_{}/".format(model_type, current_time)

model_history = train_and_evaluate(image_model, epochs, steps_per_epoch, output_dir)

Train for 50 steps, validate for 100 steps
Epoch 1/5
50/50 - 8s - loss: 1.0000 - accuracy: 0.6916 - val_loss: 0.3848 - val_accuracy: 0.8898
Epoch 2/5
50/50 - 1s - loss: 0.3793 - accuracy: 0.8858 - val_loss: 0.3499 - val_accuracy: 0.8945
Epoch 3/5
50/50 - 1s - loss: 0.3263 - accuracy: 0.9020 - val_loss: 0.2613 - val_accuracy: 0.9226
Epoch 4/5
50/50 - 1s - loss: 0.2924 - accuracy: 0.9152 - val_loss: 0.2018 - val_accuracy: 0.9390
Epoch 5/5
50/50 - 1s - loss: 0.2365 - accuracy: 0.9298 - val_loss: 0.1862 - val_accuracy: 0.9454
Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_102 (Flatten)        multiple                  0         
_________________________________________________________________
dense_254 (Dense)            multiple                  314000    
_________________________________________________________________
dense_255 (Dense)            multiple                

In [77]:
model_type = 'cnn' # linear / dnn / dnn_dropout / cnn
current_time = datetime.now().strftime("%y%m%d_%H%M%S")
epochs = 5
steps_per_epoch = 50
output_dir = "mnist_trainer/models/{}_{}/".format(model_type, current_time)

model_history = train_and_evaluate(image_model, epochs, steps_per_epoch, output_dir)

Train for 50 steps, validate for 100 steps
Epoch 1/5
50/50 - 8s - loss: 1.0630 - accuracy: 0.6692 - val_loss: 0.3409 - val_accuracy: 0.9046
Epoch 2/5
50/50 - 4s - loss: 0.3096 - accuracy: 0.9062 - val_loss: 0.1724 - val_accuracy: 0.9462
Epoch 3/5
50/50 - 4s - loss: 0.2010 - accuracy: 0.9370 - val_loss: 0.1295 - val_accuracy: 0.9576
Epoch 4/5
50/50 - 3s - loss: 0.1730 - accuracy: 0.9484 - val_loss: 0.1073 - val_accuracy: 0.9667
Epoch 5/5
50/50 - 3s - loss: 0.1489 - accuracy: 0.9532 - val_loss: 0.0904 - val_accuracy: 0.9709
Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 11, 11, 32)      